In [1]:
library(ggplot2)
library(reshape2)
library(dplyr)
library(ggrepel)
library(data.table)
library(dbscan)
library(tidyr)
library(uwot)
library(scales)
source('/disk0/sm/project/addiction/function.R')
# library(tidyverse)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following objects are masked from ‘package:reshape2’:

    dcast, melt



Attaching package: ‘dbscan’


The following object is masked from ‘package:stats’:

    as.dendrogram



Attaching package: ‘tidyr’


The following object is masked from ‘package:reshape2’:

    smiths


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




In [4]:
# exp <- fread('/disk0/sm/project/addiction/data/download_108.tsv', sep='\t', skip=222, header=TRUE, fill=TRUE)[,1:110]
exp <- fread('/disk0/sm/project/addiction/data/GDS3703_full.soft', sep='\t', skip=222, header=TRUE, fill=TRUE)[,1:110]
exp <- exp[, c("ID_REF", names(exp)[3:110]), with=FALSE]

In [8]:
# GEO GDS 파일에서 subset 정보 읽어오기
gds_file <- "/disk0/sm/project/addiction/data/GDS3703_full.soft" 
gds_lines <- readLines(gds_file)

# subset 관련 라인만 추출
subset_lines <- grep("^\\^SUBSET", gds_lines)
desc_lines   <- grep("!subset_description", gds_lines)
id_lines     <- grep("!subset_sample_id", gds_lines)
type_lines   <- grep("!subset_type", gds_lines)

# subset 정보 추출
subset_info <- data.frame(
  description = gds_lines[desc_lines] %>% gsub("!subset_description = ", "", .),
  sample_ids  = gds_lines[id_lines] %>% gsub("!subset_sample_id = ", "", .),
  type        = gds_lines[type_lines] %>% gsub("!subset_type = ", "", .),
  stringsAsFactors = FALSE
)

# sample별로 long format으로 분리
subset_long <- subset_info %>%
  separate_rows(sample_ids, sep = ",") %>%
  rename(sample = sample_ids)

# agent, time 매핑
agent_map <- subset_long %>% filter(type=="agent") %>% select(sample, agent=description)
time_map  <- subset_long %>% filter(type=="time")  %>% select(sample, time=description)

# 합치기
sample_info <- left_join(agent_map, time_map, by="sample")
sample_info$time_num <- as.numeric(gsub(" h", "", sample_info$time))
# 결과 확인
dim(sample_info)

write.table(sample_info, '/disk0/sm/project/addiction/data/sample_info.txt', sep='\t', quote=FALSE, row.names=FALSE)

Warning message in readLines(gds_file):
“incomplete final line found on '/disk0/sm/project/addiction/data/GDS3703_full.soft'”


[1] 108   4

In [9]:
## 01. violin plot으로 데이터 정규화 확인
mexp <- melt(exp, id.value='ID_REF')

p <- ggplot(mexp, aes(x = variable, y = value)) +
  geom_violin(trim = FALSE, alpha = 0.6, fill='#ffc400') +
  geom_boxplot(width = 0.2, outlier.shape = NA, alpha = 0.8) + # violin 위에 boxplot 겹쳐서 분포+요약 같이 보기
  labs(x = "sample", y = "expression") +
  theme_bw() +
  theme(
    axis.text.x = element_text(size = 8, angle = 45, hjust = 1),
    axis.text.y = element_text(size = 8),
    axis.title.x = element_text(size = 12),
    axis.title.y = element_text(size = 12),
    panel.grid.major.x = element_line(linetype = "dotted", colour = "grey70"),
    panel.grid.major.y = element_line(linetype = "dotted", colour = "grey70"),
    panel.grid.minor.x = element_blank(),
    panel.grid.minor.y = element_line(linetype = "dotted", colour = "grey70"),
    legend.position = "none"
  )

ggsave(p, file = "/disk0/sm/project/addiction/plot/01_normalizationCheck.png", width = 20000, height = 4000, units = 'px', dpi = 1000)

## 수치로도 확인
mat <- as.matrix(exp[ , -1, with=FALSE])  # ID_REF 제외
summary(mat)
apply(mat, 2, median)  # 각 샘플 median
apply(mat, 2, mean)    # 각 샘플 mean
apply(mat, 2, sd)      # 각 샘플 표준편차

Warning message in melt.data.table(exp, id.value = "ID_REF"):
“id.vars and measure.vars are internally guessed when both are 'NULL'. All non-numeric/integer/logical type columns are considered id.vars, which in this case are columns [ID_REF, ...]. Consider providing at least one of 'id' or 'measure' vars in future.”


   GSM396134        GSM396148        GSM396164        GSM396135     
 Min.   : 5.609   Min.   : 5.396   Min.   : 5.403   Min.   : 5.419  
 1st Qu.: 6.394   1st Qu.: 6.397   1st Qu.: 6.396   1st Qu.: 6.396  
 Median : 6.779   Median : 6.786   Median : 6.785   Median : 6.780  
 Mean   : 7.528   Mean   : 7.530   Mean   : 7.533   Mean   : 7.528  
 3rd Qu.: 8.130   3rd Qu.: 8.140   3rd Qu.: 8.141   3rd Qu.: 8.126  
 Max.   :15.943   Max.   :15.838   Max.   :16.081   Max.   :15.943  
   GSM396149        GSM396165        GSM396136        GSM396150     
 Min.   : 5.558   Min.   : 5.494   Min.   : 5.444   Min.   : 5.344  
 1st Qu.: 6.398   1st Qu.: 6.397   1st Qu.: 6.397   1st Qu.: 6.396  
 Median : 6.785   Median : 6.778   Median : 6.775   Median : 6.782  
 Mean   : 7.528   Mean   : 7.526   Mean   : 7.528   Mean   : 7.528  
 3rd Qu.: 8.127   3rd Qu.: 8.127   3rd Qu.: 8.131   3rd Qu.: 8.127  
 Max.   :15.829   Max.   :15.962   Max.   :15.962   Max.   :15.943  
   GSM396166        GSM396137     

GSM396134 GSM396148 GSM396164 GSM396135 GSM396149 GSM396165 GSM396136 GSM396150 
  6.77875   6.78566   6.78492   6.77973   6.78509   6.77813   6.77508   6.78221 
GSM396166 GSM396137 GSM396151 GSM396167 GSM396188 GSM396208 GSM396228 GSM396193 
  6.78439   6.77935   6.78176   6.78491   6.78301   6.77863   6.78625   6.78427 
GSM396213 GSM396233 GSM396180 GSM396184 GSM396218 GSM396195 GSM396203 GSM396223 
  6.78716   6.78465   6.77783   6.77688   6.77711   6.77950   6.78775   6.78996 
GSM396138 GSM396152 GSM396168 GSM396139 GSM396153 GSM396169 GSM396128 GSM396154 
  6.78059   6.78432   6.78385   6.78214   6.78872   6.78271   6.78114   6.78300 
GSM396170 GSM396129 GSM396155 GSM396171 GSM396192 GSM396212 GSM396232 GSM396179 
  6.77775   6.78068   6.77763   6.78642   6.78208   6.78110   6.78530   6.78384 
GSM396183 GSM396217 GSM396194 GSM396202 GSM396222 GSM396199 GSM396207 GSM396227 
  6.77531   6.78125   6.77743   6.78583   6.78981   6.78636   6.78425   6.78757 
GSM396130 GSM396156 GSM396172 GSM396131 GSM396157 GSM396173 GSM396132 GSM396158 
  6.78182   6.77911   6.78503   6.78107   6.77943   6.77846   6.77944   6.78246 
GSM396174 GSM396133 GSM396159 GSM396175 GSM396196 GSM396204 GSM396224 GSM396189 
  6.78252   6.78068   6.78081   6.77992   6.78089   6.77926   6.78665   6.78182 
GSM396209 GSM396229 GSM396176 GSM396214 GSM396234 GSM396181 GSM396185 GSM396219 
  6.78019   6.78343   6.77457   6.78543   6.78721   6.78070   6.78622   6.76417 
GSM396140 GSM396144 GSM396160 GSM396186 GSM396200 GSM396220 GSM396141 GSM396145 
  6.78382   6.78549   6.78229   6.78681   6.78414   6.78680   6.78219   6.78290 
GSM396161 GSM396197 GSM396205 GSM396225 GSM396142 GSM396146 GSM396162 GSM396190 
  6.77307   6.78400   6.77928   6.78677   6.77813   6.78472   6.77792   6.77998 
GSM396210 GSM396230 GSM396143 GSM396147 GSM396163 GSM396177 GSM396215 GSM396235 
  6.78054   6.78228   6.77945   6.78643   6.77262   6.78228   6.78535   6.78644 
GSM396178 GSM396182 GSM396216 GSM396187 GSM396201 GSM396221 GSM396198 GSM396206 
  6.77991   6.78409   6.77961   6.79033   6.78471   6.78262   6.78206   6.77872 
GSM396226 GSM396191 GSM396211 GSM396231 
  6.78214   6.78317   6.78050   6.77098

GSM396134 GSM396148 GSM396164 GSM396135 GSM396149 GSM396165 GSM396136 GSM396150 
 7.527900  7.529869  7.532607  7.527921  7.528333  7.525768  7.527705  7.528341 
GSM396166 GSM396137 GSM396151 GSM396167 GSM396188 GSM396208 GSM396228 GSM396193 
 7.528493  7.526938  7.528682  7.534036  7.528293  7.527409  7.530823  7.529642 
GSM396213 GSM396233 GSM396180 GSM396184 GSM396218 GSM396195 GSM396203 GSM396223 
 7.531029  7.529173  7.527440  7.528765  7.528940  7.526520  7.529508  7.532078 
GSM396138 GSM396152 GSM396168 GSM396139 GSM396153 GSM396169 GSM396128 GSM396154 
 7.529296  7.528417  7.533625  7.528012  7.530077  7.533394  7.526598  7.530462 
GSM396170 GSM396129 GSM396155 GSM396171 GSM396192 GSM396212 GSM396232 GSM396179 
 7.524585  7.527530  7.529726  7.533578  7.528579  7.531500  7.530782  7.527900 
GSM396183 GSM396217 GSM396194 GSM396202 GSM396222 GSM396199 GSM396207 GSM396227 
 7.527194  7.530617  7.527917  7.530008  7.531846  7.529355  7.529807  7.529322 
GSM396130 GSM396156 GSM396172 GSM396131 GSM396157 GSM396173 GSM396132 GSM396158 
 7.528409  7.529309  7.528028  7.527619  7.528542  7.525292  7.527832  7.528812 
GSM396174 GSM396133 GSM396159 GSM396175 GSM396196 GSM396204 GSM396224 GSM396189 
 7.533104  7.527648  7.529682  7.529117  7.527203  7.529951  7.530337  7.527647 
GSM396209 GSM396229 GSM396176 GSM396214 GSM396234 GSM396181 GSM396185 GSM396219 
 7.529244  7.529361  7.526247  7.529544  7.530026  7.529217  7.528247  7.525244 
GSM396140 GSM396144 GSM396160 GSM396186 GSM396200 GSM396220 GSM396141 GSM396145 
 7.530191  7.531247  7.528170  7.529698  7.530848  7.530790  7.533002  7.526480 
GSM396161 GSM396197 GSM396205 GSM396225 GSM396142 GSM396146 GSM396162 GSM396190 
 7.524699  7.527769  7.528440  7.530936  7.531234  7.530239  7.526789  7.526150 
GSM396210 GSM396230 GSM396143 GSM396147 GSM396163 GSM396177 GSM396215 GSM396235 
 7.528647  7.527863  7.527804  7.529944  7.522647  7.528808  7.530478  7.526996 
GSM396178 GSM396182 GSM396216 GSM396187 GSM396201 GSM396221 GSM396198 GSM396206 
 7.527632  7.529443  7.528867  7.529304  7.531146  7.530854  7.527997  7.528046 
GSM396226 GSM396191 GSM396211 GSM396231 
 7.527042  7.528442  7.528566  7.523187

GSM396134 GSM396148 GSM396164 GSM396135 GSM396149 GSM396165 GSM396136 GSM396150 
 1.684317  1.683711  1.689477  1.684605  1.684918  1.682340  1.686328  1.686328 
GSM396166 GSM396137 GSM396151 GSM396167 GSM396188 GSM396208 GSM396228 GSM396193 
 1.684493  1.685995  1.688333  1.691545  1.688314  1.687829  1.685795  1.686861 
GSM396213 GSM396233 GSM396180 GSM396184 GSM396218 GSM396195 GSM396203 GSM396223 
 1.685654  1.685745  1.689690  1.682395  1.692449  1.686292  1.686392  1.686053 
GSM396138 GSM396152 GSM396168 GSM396139 GSM396153 GSM396169 GSM396128 GSM396154 
 1.686852  1.687540  1.689718  1.686905  1.687970  1.690926  1.685102  1.688838 
GSM396170 GSM396129 GSM396155 GSM396171 GSM396192 GSM396212 GSM396232 GSM396179 
 1.683197  1.685916  1.688154  1.693395  1.686114  1.692285  1.692215  1.689279 
GSM396183 GSM396217 GSM396194 GSM396202 GSM396222 GSM396199 GSM396207 GSM396227 
 1.681325  1.687634  1.682396  1.687880  1.687896  1.688936  1.686123  1.687859 
GSM396130 GSM396156 GSM396172 GSM396131 GSM396157 GSM396173 GSM396132 GSM396158 
 1.684738  1.688031  1.683599  1.686008  1.687359  1.683023  1.684620  1.686895 
GSM396174 GSM396133 GSM396159 GSM396175 GSM396196 GSM396204 GSM396224 GSM396189 
 1.690875  1.685148  1.689494  1.688427  1.686510  1.686773  1.685262  1.688374 
GSM396209 GSM396229 GSM396176 GSM396214 GSM396234 GSM396181 GSM396185 GSM396219 
 1.687732  1.684649  1.682093  1.690889  1.688167  1.688422  1.683114  1.683283 
GSM396140 GSM396144 GSM396160 GSM396186 GSM396200 GSM396220 GSM396141 GSM396145 
 1.687170  1.684412  1.687281  1.687938  1.687765  1.689345  1.689383  1.683604 
GSM396161 GSM396197 GSM396205 GSM396225 GSM396142 GSM396146 GSM396162 GSM396190 
 1.684752  1.687398  1.684726  1.685478  1.689702  1.685126  1.689785  1.687133 
GSM396210 GSM396230 GSM396143 GSM396147 GSM396163 GSM396177 GSM396215 GSM396235 
 1.686825  1.687585  1.687018  1.684371  1.689532  1.685730  1.683686  1.684840 
GSM396178 GSM396182 GSM396216 GSM396187 GSM396201 GSM396221 GSM396198 GSM396206 
 1.689167  1.691368  1.686578  1.686297  1.689828  1.683729  1.687245  1.690626 
GSM396226 GSM396191 GSM396211 GSM396231 
 1.686906  1.687471  1.685681  1.680293

In [10]:
## 02. 분산 큰 gene 선별
txp <- t(as.matrix(exp[, -1, with=FALSE]))
colnames(txp) <- exp$ID_REF

col_var <- apply(txp, 2, var)  # 각 열(probe)별 분산
summary(col_var)                # 최소, 1사분위, 중앙값, 평균, 최대 확인

df_var <- data.frame(probe = names(col_var), variance = as.numeric(col_var)) %>% arrange(desc(variance))

## 분산 막대그래프 그릴 것
p <- ggplot(df_var[1:600,], aes(x = reorder(probe, -variance), y = variance)) +
  geom_bar(stat="identity", alpha = 0.6, fill='#ffc400') +
  labs(x = "sample", y = "variant") +
  theme_bw() +
  theme(
    axis.text.x = element_text(size = 8, angle = 45, hjust = 1),
    axis.text.y = element_text(size = 8),
    axis.title.x = element_text(size = 12),
    axis.title.y = element_text(size = 12),
    panel.grid.major.x = element_line(linetype = "dotted", colour = "grey70"),
    panel.grid.major.y = element_line(linetype = "dotted", colour = "grey70"),
    panel.grid.minor.x = element_blank(),
    panel.grid.minor.y = element_line(linetype = "dotted", colour = "grey70"),
    legend.position = "none"
  )

ggsave(p, file = "/disk0/sm/project/addiction/plot/02_variantGeneSelect.png", width = 20000, height = 4000, units = 'px', dpi = 1000)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
0.008503 0.020443 0.029337 0.042709 0.047630 2.628735 

In [11]:
## 03. target 선정
annotation_colors <- list(
  agent = c(
      "cocaine"="#db1f1f", 
      "ethanol"="#2B3499", 
      "heroin"="#FF9209", 
      "methamphetamine"='#6C22FF',
      'morphine'='#22FF6C',
      'nicotine'='grey30',
      'saline'='grey'
  )
)

In [13]:
## 약물 설정(여기서 조정해주기!!)
addiction_list <- unique(sample_info$agent) # c("cocaine", "ethanol", "heroin", "methamphetamine", "morphine", "nicotine")     "saline" "none"  
set.seed(123)

feature <- 500  # 200   500    5000   46632
time <- c(1, 2, 4, 8)

## 반복!
# for (i in 1:6) {
    # target_addiction <- addiction_list[i]
    target_addiction <- c("cocaine", "ethanol", "heroin", "methamphetamine", "morphine", "nicotine")

    if (all(target_addiction == c("cocaine", "ethanol", "heroin", "methamphetamine", "morphine", "nicotine"))) {
        key <- ''
    } else {
        key <- target_addiction
    }

    ## 분산으로 target gene 설정, 약물 관련 사람만 뽑기
    target_result <- target_select(sample_info, df_var, feature, c('saline', target_addiction), time)
    txp_top <- target_result[[1]]
    target <- target_result[[2]]
    dim(txp_top)

    # 03. PCA 수행
    PCA_result <- run_PCA(txp_top)
    run_PCAplot(PCA_result, annotation_colors, key)
    run_PCAdbscan(PCA_result, feature, key)

    ## 04. UMAP 수행
    UMAP_result <- run_UMAP(txp_top, target)
    run_UMAPplot(UMAP_result, annotation_colors, key)
    run_UMAPdbscan(UMAP_result, feature, key)
# }

[1]  96 500

PC4 
  4 


In [19]:
summary(PCA_result[[4]])

Importance of components:
                           PC1    PC2    PC3     PC4     PC5     PC6     PC7
Standard deviation     14.5668 8.4717 7.0743 5.25165 3.54576 3.45223 3.44403
Proportion of Variance  0.4244 0.1435 0.1001 0.05516 0.02514 0.02384 0.02372
Cumulative Proportion   0.4244 0.5679 0.6680 0.72317 0.74832 0.77215 0.79588
                           PC8     PC9    PC10    PC11    PC12    PC13    PC14
Standard deviation     3.14374 2.92059 2.59978 2.43733 2.26050 2.15268 2.10498
Proportion of Variance 0.01977 0.01706 0.01352 0.01188 0.01022 0.00927 0.00886
Cumulative Proportion  0.81564 0.83270 0.84622 0.85810 0.86832 0.87759 0.88645
                          PC15    PC16   PC17    PC18    PC19    PC20    PC21
Standard deviation     1.85351 1.82416 1.7025 1.61675 1.55659 1.44628 1.39407
Proportion of Variance 0.00687 0.00666 0.0058 0.00523 0.00485 0.00418 0.00389
Cumulative Proportion  0.89332 0.89998 0.9058 0.91100 0.91585 0.92003 0.92392
                          PC22    PC23

In [ ]:
######################################################

In [ ]:
## 04. 계층적 클러스터링
library(pheatmap)
library(dplyr)
library(RColorBrewer)
library(grid)


txp_top_t <- t(txp_top)

# 1. annotation_col 생성
annotation_col <- target %>% select(sample, agent) %>% tibble::column_to_rownames("sample")


# annotation_col rownames와 txp_top 열 이름이 겹치는 것만 필터링
common_samples <- intersect(rownames(annotation_col), colnames(txp_top_t))

# 순서 맞추기
txp_top_t <- txp_top_t[, common_samples]
annotation_col <- annotation_col[common_samples, , drop = FALSE]

# 확인
all(rownames(annotation_col) == colnames(txp_top_t))

# # 2. annotation 색상 지정
# annotation_colors <- list(
#   agent = c(
#     "cocaine"="#FF6C22", 
#     "ethanol"="#2B3499", 
#     "heroin"="#FF9209", 
#     "methamphetamine"='#6C22FF',
#     'morphine'='#22FF6C',
#     'nicotine'='grey30',
#     'saline'='grey'
#   )
# )

# 3. Heatmap 생성
p <- pheatmap(
  txp_top_t,
  clustering_method = "ward.D2",  # ward.D2
  show_rownames = FALSE,
  show_colnames = TRUE,
  annotation_col = annotation_col,
  annotation_colors = annotation_colors,
  color = colorRampPalette(c("#6E7582","#ffffff","#ffd369"))(100)
)

# 4. 저장 함수
save_pheatmap_png <- function(x, filename, width = 8000, height = 8000, res = 1000, units = "px") {
  png(filename, width = width, height = height, res = res, units = units)
  grid::grid.newpage()
  grid::grid.draw(x$gtable)
  dev.off()
}

save_pheatmap_png(p, paste0("/disk0/sm/project/addiction/plot/04_heatmap", feature, ".png"))

In [ ]:
#############################################
#############################################
#############################################

In [ ]:
annotation_colors <- list(
  agent = c(
      "cocaine"="#ed00f5", 
      "ethanol"="#9410ff", 
      "heroin"="#FF9209", 
      "methamphetamine"='#ffcb22',
      'morphine'='#ff6600',
      'nicotine'='#ff1313',
      'saline'='#333333', 
      'none'='blue'
  ),
  time = c(
    "1 h"="#ffcb22", 
    '2 h'='#FF9209', 
    '4 h'='#ff6600', 
    '8 h'='#ff1313'
  )
)


In [ ]:
# time <- c(1, 2, 4, 8)
# feature <- 50  # 3032
# target_result <- target_select(sample_info, df_var, feature, c('none', 'saline', target_addiction), time)
# txp_top <- target_result[[1]]
# target <- target_result[[2]]

# ### PCA
# pca_res <- prcomp(txp_top, scale.=TRUE)
# pca_var <- summary(pca_res)$importance[2,]
# cum_var <- cumsum(pca_var)  # 누적 분산
# num_pc <- which(cum_var >= 0.7)[1]  # 70% 이상 설명하는 최소 PC 수

# print(num_pc)

# pc_for_umap <- pca_res$x[, 1:num_pc]

# umap_res <- umap(pc_for_umap, n_neighbors = 5, min_dist = 0.1, metric = "euclidean")
# umap_df <- data.frame(UMAP1 = umap_res[,1], UMAP2 = umap_res[,2], agent = target$agent, time_num = target$time_num)


# p <- ggplot(umap_df, aes(x = UMAP1, y = UMAP2, color = agent, size = time_num)) +
# geom_point() +      # alpha도 time_num에 따라
# scale_size_continuous(range = c(2, 6)) +  # 점 크기: 1h~8h
# # scale_alpha_continuous(range = c(0.3, 1)) +  # 점 진하기: 1h 연하게, 8h 진하게
# scale_color_manual(values = annotation_colors$agent) + # agent별 색상 적용
# labs(
#     x = "UMAP1",
#     y = "UMAP2",
#     color = "Agent",
#     size = "Time (h)",
#     alpha = "Time (h)"
# ) +
# theme_bw() +
# theme(
#     axis.text.x = element_text(size = 8, angle = 45, hjust = 1),
#     axis.text.y = element_text(size = 8),
#     axis.title.x = element_text(size = 12),
#     axis.title.y = element_text(size = 12),
#     panel.grid.major.x = element_line(linetype = "dotted", colour = "grey70"),
#     panel.grid.major.y = element_line(linetype = "dotted", colour = "grey70"),
#     panel.grid.minor.x = element_blank(),
#     panel.grid.minor.y = element_line(linetype = "dotted", colour = "grey70")
# )

# ggsave(p, file = paste0("/disk0/sm/project/addiction/plot/03_UMAP_", feature, ".png"), width = 8000, height = 6000, units = 'px', dpi = 1000)

In [ ]:
time <- c(1, 2, 4, 8)
feature <- 2013  # 3032 2013
target_result <- target_select(sample_info, df_var, feature, c('saline', target_addiction), time)
txp_top <- target_result[[1]]
target <- target_result[[2]]

### PCA
pca_res <- prcomp(txp_top, scale.=TRUE)
pca_var <- summary(pca_res)$importance[2,]
# cum_var <- cumsum(pca_var)  # 누적 분산
# num_pc <- which(cum_var >= 0.7)[1]  # 70% 이상 설명하는 최소 PC 수

# print(num_pc)
num_pc <- 50
pc_for_umap <- pca_res$x[, 1:num_pc]

umap_res <- umap(pc_for_umap, n_neighbors = 5, min_dist = 0.1, metric = "euclidean")
umap_df <- data.frame(UMAP1 = umap_res[,1], UMAP2 = umap_res[,2], agent = target$agent, time = target$time)


p <- ggplot(umap_df, aes(x = UMAP1, y = UMAP2, color = time)) +
geom_point() +      # alpha도 time_num에 따라
scale_color_manual(values = annotation_colors$time) + # agent별 색상 적용
labs(
    x = "UMAP1",
    y = "UMAP2",
    color = "Agent",
    size = "Time (h)",
    alpha = "Time (h)"
) +
theme_bw() +
theme(
    axis.text.x = element_text(size = 8, angle = 45, hjust = 1),
    axis.text.y = element_text(size = 8),
    axis.title.x = element_text(size = 12),
    axis.title.y = element_text(size = 12),
    panel.grid.major.x = element_line(linetype = "dotted", colour = "grey70"),
    panel.grid.major.y = element_line(linetype = "dotted", colour = "grey70"),
    panel.grid.minor.x = element_blank(),
    panel.grid.minor.y = element_line(linetype = "dotted", colour = "grey70")
)

ggsave(p, file = paste0("/disk0/sm/project/addiction/plot/03_UMAP_", feature, ".png"), width = 8000, height = 6000, units = 'px', dpi = 1000)

In [ ]:


# summary(pca_res)          # 각 PC 설명분산 확인
# head(pca_res$x)           # 샘플별 PC 좌표

# var_explained <- (pca_res$sdev^2) / sum(pca_res$sdev^2)

# # PC1, PC2 각각
# PC1_var <- round(var_explained[1] * 100, 2)
# PC2_var <- round(var_explained[2] * 100, 2)

# pca_result <- data.frame(pca_res$x)
# pca_result$sample <- row.names(pca_result)
# pca_df <- merge(pca_result, sample_info, by='sample', all.x=TRUE)



# p <- ggplot(pca_df, aes(x = PC1, y = PC2, color = agent, size = time_num)) +
# geom_point(aes(alpha = time_num)) +      # alpha도 time_num에 따라
# scale_size_continuous(range = c(2, 6)) +  # 점 크기: 1h~8h
# # scale_alpha_continuous(range = c(0.3, 1)) +  # 점 진하기: 1h 연하게, 8h 진하게
# scale_color_manual(values = annotation_colors$agent) + # agent별 색상 적용
# labs(
#     x = paste0("PC1 (", PC1_var, "%)"),
#     y = paste0("PC2 (", PC2_var, "%)"),
#     color = "Agent",
#     size = "Time (h)",
#     alpha = "Time (h)"
# ) +
# theme_bw() +
# theme(
#     axis.text.x = element_text(size = 8, angle = 45, hjust = 1),
#     axis.text.y = element_text(size = 8),
#     axis.title.x = element_text(size = 12),
#     axis.title.y = element_text(size = 12),
#     panel.grid.major.x = element_line(linetype = "dotted", colour = "grey70"),
#     panel.grid.major.y = element_line(linetype = "dotted", colour = "grey70"),
#     panel.grid.minor.x = element_blank(),
#     panel.grid.minor.y = element_line(linetype = "dotted", colour = "grey70")
# )


# ggsave(p, file = paste0("/disk0/sm/project/addiction/plot/03_PCA_", feature, ".png"), width = 8000, height = 6000, units = 'px', dpi = 1000)
